In [1]:
import joblib
import os
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import pickle
import time
from tkinter import *
import tkinter as tk
from tkinter import filedialog
from tqdm import tqdm
from tqdm import notebook
import itertools
import math
import seaborn as sns

# Selecting Multiple NumPy Files

In [2]:
# Create Tk root
root = tk.Tk()
# Hide the main window
root.withdraw()
root.call('wm', 'attributes', '.', '-topmost', True)

# Ask user to select multiple files from different folders
infiles = []
while True:
    file = filedialog.askopenfilename(title='Load posthoc-predictions.npy', initialdir='/')
    if not file:  # Break the loop if no file selected or Cancel button clicked
        break
    infiles.append(file)

# Iterate over selected files
for file in infiles:
    print('Selected File:', file)

Selected File: Z:/KayCei/VideoOutputCamera 0-09052023-123830_posthoc-predictions.npy
Selected File: Z:/KayCei/VideoOutputCamera 0-11052023-154826_posthoc-predictions.npy
Selected File: Z:/KayCei/AD11-D1/51123-rt bilateral/VideoOutputCamera 0-11052023-143405_posthoc-predictions.npy
Selected File: Z:/KayCei/AD6-D2/rt bilateral/VideoOutputCamera 0-09052023-123830_posthoc-predictions.npy


# Total Time of Video in deciseconds

In [3]:
# Create an empty dictionary to store random lists separately
behavior_dict = {}

# Iterate over selected files
for file in infiles:
    # Load the file as a NumPy array
    behavior = np.load(file)
    
    # Get the file name without the extension
    file_name = os.path.splitext(os.path.basename(file))[0]
    
    # Check if the file name already exists in the dictionary
    if file_name in behavior_dict:
        # If the file name exists, append the array to the existing list
        behavior_dict[file_name].append(list(behavior))
    else:
        # If the file name doesn't exist, create a new list with the array
        behavior_dict[file_name] = [list(behavior)]

    print('This video is', len(behavior), 'deciseconds, which is', (len(behavior) / 600), 'minutes')
    

This video is 39779 deciseconds, which is 66.29833333333333 minutes
This video is 39779 deciseconds, which is 66.29833333333333 minutes
This video is 42557 deciseconds, which is 70.92833333333333 minutes
This video is 44935 deciseconds, which is 74.89166666666667 minutes


In [5]:
#Print the random lists separately using the dictionary
for file_name, random_list in behavior_dict.items():
    print(f'\033[1;31mFile: {file_name}\033[0m')
    for behavior in random_list:
        print(f'Random List: {behavior}')
    print('\u2501' * 50)  # Print a wider black line consisting of the Unicode character U+2501
    print()


File: VideoOutputCamera 0-09052023-123830_posthoc-predictions
Random List: [5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 3.0, 3.0, 3.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 0.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 0.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0, 2.0,

# Creates an array with tuples of (behavior type, stim count)

In [6]:
behavior_type_and_duration_dict = {}

# Iterate over each key-value pair in the behavior_dict
for file_name, behavior_list in behavior_dict.items():
    behavior_type_and_duration = []
    for behavior in behavior_list:
        current_value = behavior[0]
        count = 0
        for value in behavior:
            if value == current_value:
                count += 1
            else:
                behavior_type_and_duration.append((current_value, count))
                current_value = value
                count = 1
        
        # Append the final (current_value, count) pair outside the loop
        behavior_type_and_duration.append((current_value, count))
        
    # Store the behavior_type_and_duration list in the behavior_type_and_duration_dict
    behavior_type_and_duration_dict[file_name] = behavior_type_and_duration
    


# Filtering Out Based on Frame Rate of 3

In [7]:
for file_name, behavior_type_and_duration in behavior_type_and_duration_dict.items():
    filtered_behavior_type_and_duration = [(behavior, duration) for behavior, duration in behavior_type_and_duration if duration >= 3]
    behavior_type_and_duration_dict[file_name] = filtered_behavior_type_and_duration


# Setting Behavior of Interest

Trigger key 
0 Left Turns 
1 Right Turns 
2 Locomotion 
3 Face Groom 
4 All Other Groom 
5 All Other 

In [8]:
behavior_of_interest= 2

#  Total Bouts

In [9]:
count = 0  # Set counter to zero

for behavior_type_and_duration in behavior_type_and_duration_dict.values():
    for behavior, duration in behavior_type_and_duration:
        if behavior == behavior_of_interest:
            count += 1

print(count, "bout(s) of behavior", behavior_of_interest)  # Print the total number of bouts found

2815 bout(s) of behavior 2


# Average Duration and Count 

In [11]:
Duration_of_behavior_type = []  # Empty list to store the durations of the behavior type

print("Available File Names:")
for file_name in behavior_type_and_duration_dict:
    print(file_name)

# Ask the user for their choice
choice = input("Enter 'A' to calculate for all files, or 'I' for individual files: ")

if choice.upper() == 'A':
    # Calculate for all files
    num_of_stims = 0
    count = 0
    average_duration = []

    for behavior_type_and_duration in behavior_type_and_duration_dict.values():
        stim_sum = sum(duration for behavior, duration in behavior_type_and_duration if behavior == behavior_of_interest)
        num_of_stims += stim_sum
        count += 1

        from statistics import mean
        for behavior_type_and_duration in behavior_type_and_duration_dict.values():
            durations = [duration for behavior, duration in behavior_type_and_duration if behavior == behavior_of_interest]
            if durations:  # Check if there are durations for the behavior of interest
                average_individual_duration = mean(durations)
                average_duration.append(average_individual_duration)

    stim_count = num_of_stims
    print("Behavior", behavior_of_interest, "was stimmed", stim_count, "times")

    if count > 0:
        average_stim_count = num_of_stims / count
        print("The average stim count of behavior", behavior_of_interest, "is",  average_stim_count)
        
        from statistics import mean

        if len(average_duration) > 0:
            overall_average = mean(average_duration)
            print("The average duration of behavior", behavior_of_interest, "is", overall_average)
        else:
            print("No occurrences of behavior", behavior_of_interest, "found.")

elif choice.upper() == 'I':
    # Calculate for individual files
    # Ask the user to select a file name
    selected_file_name = input("Select a File Name: ")

    # Check if the selected file name exists in the behavior_type_and_duration_dict
    if selected_file_name in behavior_type_and_duration_dict:
        # Select the behavior type and duration list from the dictionary
        behavior_type_and_duration_list = behavior_type_and_duration_dict[selected_file_name]
        for behavior, duration in behavior_type_and_duration_list:
            if behavior == behavior_of_interest:
                Duration_of_behavior_type.append(duration)

        stim_count = sum(Duration_of_behavior_type)
        print("Behavior", behavior_of_interest, "was stimmed", stim_count, "times")

        from statistics import mean
        duration_average = mean(Duration_of_behavior_type)
        print("The average duration of behavior", behavior_of_interest, "is", duration_average)

    else:
        print("Invalid file name. Please select a valid file name from the list.")
else:
    print("Invalid choice. Please enter 'A' or 'I' to choose the calculation type.")


Available File Names:
VideoOutputCamera 0-09052023-123830_posthoc-predictions
VideoOutputCamera 0-11052023-154826_posthoc-predictions
VideoOutputCamera 0-11052023-143405_posthoc-predictions
Enter 'A' to calculate for all files, or 'I' for individual files: I
Select a File Name: VideoOutputCamera 0-09052023-123830_posthoc-predictions
Behavior 2 was stimmed 9141 times
The average duration of behavior 2 is 5.709556527170519


# WORKS BUT CHANGED IT